# Langchain

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

## LCEL (LangChain Expression Language)
- LangChain 重写了 `|`（`__or__`），`Chain 之所在`\n",
- `RunnablePassthrough`: RunnablePassthrough 允许你将输入数据直接传递而不做任何更改（identity），通常与 RunnableParallel 一起使用，将数据传递到新的键中。\n",
- LangChain 的应用 RunnablePassthrough 作为一个占位符，可以在需要时填充数据，比如在公司名称尚未确定时先留空，后续再填入。\n",
- 所谓的最佳实践\n",
    - python：遍历，也可以用 list comprehension\n",
    - 对于 matlab：也可以遍历，也可以整理成 matrix，直接矩阵矢量乘法；

In [2]:
2 | 3 > 2

True

In [3]:
2 | 3

3

In [4]:
from langchain_core.runnables import (
    RunnablePassthrough, 
    RunnableLambda, 
    RunnableParallel
)
os.environ["LANGCHAIN_PROJECT"] = 'lcel_test'

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
output_parser = StrOutputParser()
llm = ChatOpenAI(model="gpt-3.5-turbo")

# lcel
chain = (
    {"topic": RunnablePassthrough()} 
    | prompt
    | llm
    | output_parser
)

chain.invoke("ice cream")


"Why did the ice cream truck break down? It couldn't handle the rocky road ahead!"